# Test Set

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.externals import joblib
from my_measures import BinaryClassificationPerformance
%matplotlib inline

### Read raw test data

In [2]:
amazon = pd.read_csv('/Users/andrewlevinson/programs/parsons/spring-2019/machine-learning-class/machine-learning/assignment-1/data/raw_data_test.csv')

print(amazon.shape)

(91000, 14)


In [3]:
print(amazon.loc[0, "Text"])

As much as I love Italian Sweet Creme in liquid form ( the only one I use ) , this one is not even closer to the taste of the liquid, I have strange after taste like chemical or something , plus I get flux-reflect if I use to often , very disappointed and will not purchase again .


In [4]:
print(amazon.head())
print(amazon['helpful'].mean())

   Unnamed: 0  Unnamed: 0.1      Id   ProductId          UserId  \
0      450430        317629  317630  B002BUD5TK   AZ4EUUJVC8358   
1      209527        289838  289839  B003LPOUZM  A1BEU8FCL6JI4T   
2      329547        479010  479011  B001E5E074  A1XL72EEATTQY4   
3       46044        256379  256380  B001EQ55R2  A38XAAS5U05GY8   
4      162804        355065  355066  B001EQ4OBA   A6S4BCIF87GQ7   

                     ProfileName  HelpfulnessNumerator  \
0                          Kasia                     1   
1                 Linda "ln5229"                     0   
2  Michael J. Pruett "M P buyer"                     2   
3                         Judith                     0   
4            Reason "books_easy"                     1   

   HelpfulnessDenominator  Score        Time  \
0                       1      1  1340755200   
1                       0      5  1298851200   
2                       2      3  1343692800   
3                       0      5  1320451200   
4       

### Feature extraction on natural language data

In [5]:
# # http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer = CountVectorizer()
# corpus = amazon.Text.as_matrix()
# X_bag_of_words = vectorizer.fit_transform(corpus)
# print(X_bag_of_words.toarray())

In [6]:
# vectorize Bag of Words from review text; as sparse matrix
# from sklearn.feature_extraction.text import HashingVectorizer
# hv = HashingVectorizer(n_features=2 ** 17, non_negative=True)
hv = joblib.load('hv.pkl')
X_hv = hv.transform(amazon.Text)
print(X_hv.shape)

/Users/andrewlevinson/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/hashing.py:102: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


(91000, 131072)


In [7]:
# http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
# from sklearn.feature_extraction.text import TfidfTransformer
# transformer = TfidfTransformer()
transformer = joblib.load('transformer.pkl')
X_tfidf = transformer.transform(X_hv)

In [8]:
print(type(X_tfidf))

<class 'scipy.sparse.csr.csr_matrix'>


### Create additional quantitative features

In [9]:
# features from Amazon.csv to add to feature set
# review word length
amazon['reviewLen'] = amazon['Text'].apply(lambda x: len(str(x).split(" ")))

# summary word length
amazon['sumLen'] = amazon['Summary'].apply(lambda x: len(str(x).split(" ")))

# upper case words in Text review
amazon['upper'] = amazon['Text'].apply(lambda x: len([x for x in x.split() if x.isupper()]))


X_quant_features = amazon[["Score", "reviewLen", "Time", "sumLen", "upper"]]
print(X_quant_features.head(10))
# print(type(X_quant_features))

   Score  reviewLen        Time  sumLen  upper
0      1         59  1340755200       3      5
1      5         29  1298851200       3      0
2      3         23  1343692800       1      2
3      5         42  1320451200       4      1
4      3         48  1290384000       6      4
5      1        116  1251504000       3      5
6      4         18  1214092800       3      0
7      5         61  1347667200       5      7
8      5         92  1318118400       3      0
9      1         62  1340150400       4      5


### Combine all quantitative features into a single sparse matrix

In [10]:
from scipy.sparse import csr_matrix, hstack
X_quant_features_csr = csr_matrix(X_quant_features)
X_combined = hstack([X_tfidf, X_quant_features_csr])
X_matrix = csr_matrix(X_combined) # convert to sparse matrix
print(X_matrix.shape)

(91000, 131077)


### Create `X`, scaled matrix of features

In [11]:
# feature scaling
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler(with_mean=False)
sc = joblib.load('sc.pkl')
X = sc.transform(X_matrix)
print(X.shape)

(91000, 131077)


### create `y`, vector of Labels

In [12]:
y = amazon['helpful'].values
print(type(y))

<class 'numpy.ndarray'>


### predict `y` with the fit on the training set

In [13]:
# MODEL: ordinary least squares
# from sklearn import linear_model
# ols = linear_model.SGDClassifier(loss="squared_loss")
# ols.fit(X, y)
# joblib.dump(ols, 'ols.pkl') # pickle
ols = joblib.load('ols.pkl') # pickle

ols_performance = BinaryClassificationPerformance(ols.predict(X), y, 'ols')
ols_performance.compute_measures()
print(ols_performance.performance_measures)

{'Pos': 6708, 'Neg': 84292, 'TP': 3875, 'TN': 32922, 'FP': 51370, 'FN': 2833, 'Accuracy': 0.4043626373626374, 'Precision': 0.07014209430717712, 'Recall': 0.5776684555754323, 'desc': 'ols'}


In [14]:
# MODEL: SVM, linear
# from sklearn import linear_model
# svm = linear_model.SGDClassifier()
# svm.fit(X, y)
# joblib.dump(svm, 'svm.pkl') # pickle
svm = joblib.load('svm.pkl') # pickle

svm_performance = BinaryClassificationPerformance(svm.predict(X), y, 'svm')
svm_performance.compute_measures()
print(svm_performance.performance_measures)

FileNotFoundError: [Errno 2] No such file or directory: 'svm.pkl'

In [15]:
# MODEL: logistic regression
# from sklearn import linear_model
# lgs = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
# lgs.fit(X, y)
# joblib.dump(lgs, 'lgs.pkl') # pickle
lgs = joblib.load('lgs.pkl')

lgs_performance = BinaryClassificationPerformance(lgs.predict(X), y, 'lgs')
lgs_performance.compute_measures()
print(lgs_performance.performance_measures)

{'Pos': 6663, 'Neg': 84337, 'TP': 1689, 'TN': 79210, 'FP': 5127, 'FN': 4974, 'Accuracy': 0.889, 'Precision': 0.24779929577464788, 'Recall': 0.2534894191805493, 'desc': 'lgs'}


In [18]:
# MODEL: Naive Bayes
# from sklearn.naive_bayes import MultinomialNB
# nbs = MultinomialNB()
# nbs.fit(X, y)
# joblib.dump(nbs, 'nbs.pkl') # pickle
nbs = joblib.load('nbs.pkl')

nbs_performance = BinaryClassificationPerformance(nbs.predict(X), y, 'nbs')
nbs_performance.compute_measures()
print(nbs_performance.performance_measures)

{'Pos': 6708, 'Neg': 84292, 'TP': 2587, 'TN': 72171, 'FP': 12121, 'FN': 4121, 'Accuracy': 0.8215164835164835, 'Precision': 0.17589067174326897, 'Recall': 0.3856589147286822, 'desc': 'nbs'}


In [17]:
# MODEL: Perceptron
# from sklearn import linear_model
# prc = linear_model.SGDClassifier(loss='perceptron')
# prc.fit(X, y)
# joblib.dump(prc, 'prc.pkl') # pickle
prc = joblib.load('prc.pkl')

prc_performance = BinaryClassificationPerformance(prc.predict(X), y, 'prc')
prc_performance.compute_measures()
print(prc_performance.performance_measures)

{'Pos': 6663, 'Neg': 84337, 'TP': 1684, 'TN': 78706, 'FP': 5631, 'FN': 4979, 'Accuracy': 0.8834065934065934, 'Precision': 0.23021189336978812, 'Recall': 0.25273900645354946, 'desc': 'prc'}


In [15]:
# MODEL: Ridge Regression Classifier
# from sklearn import linear_model
# rdg = linear_model.RidgeClassifier()
# rdg.fit(X, y)
# joblib.dump(rdg, 'rdg.pkl') # pickle
rdg = joblib.load('rdg.pkl')

rdg_performance = BinaryClassificationPerformance(rdg.predict(X), y, 'rdg')
rdg_performance.compute_measures()
print(rdg_performance.performance_measures)

{'Pos': 6708, 'Neg': 84292, 'TP': 997, 'TN': 83019, 'FP': 1273, 'FN': 5711, 'Accuracy': 0.9232527472527473, 'Precision': 0.4392070484581498, 'Recall': 0.14862850327966606, 'desc': 'rdg'}


In [16]:
# MODEL: Decision Tree Classifier
# from sklearn import tree
# dtc = tree.DecisionTreeClassifier(max_depth=100)
# dtc.fit(X_matrix, y)
# joblib.dump(dtc, 'dtc.pkl') # pickle
dtc = joblib.load('dtc.pkl')

dtc_performance = BinaryClassificationPerformance(dtc.predict(X_matrix), y, 'dtc')
dtc_performance.compute_measures()
print(dtc_performance.performance_measures)

{'Pos': 6708, 'Neg': 84292, 'TP': 2350, 'TN': 82406, 'FP': 1886, 'FN': 4358, 'Accuracy': 0.9313846153846154, 'Precision': 0.5547686496694996, 'Recall': 0.35032796660703636, 'desc': 'dtc'}


### ROC plot to compare performance of various models and fits

In [19]:
# fits = [svm_performance, lgs_performance, nbs_performance, rdg_performance, prc_performance, ols_performance, dtc_performance]
fits = [nbs_performance, rdg_performance, ols_performance, dtc_performance]

for fit in fits:
    plt.plot(fit.performance_measures['FP'] / fit.performance_measures['Neg'], 
             fit.performance_measures['TP'] / fit.performance_measures['Pos'], 'bo')
    plt.text(fit.performance_measures['FP'] / fit.performance_measures['Neg'], 
             fit.performance_measures['TP'] / fit.performance_measures['Pos'], fit.desc)
plt.axis([0, 1, 0, 1])
plt.title('ROC plot: test set')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.show()

### looking at reviews based on their classification

Let's say we decide that Ridge Regression is the best model for generalization. Let's take a look at some of the reviews and try to make a (subjective) determination of whether it's generalizing well. 

In [18]:
amazon.loc[0, "Text"]

'As much as I love Italian Sweet Creme in liquid form ( the only one I use ) , this one is not even closer to the taste of the liquid, I have strange after taste like chemical or something , plus I get flux-reflect if I use to often , very disappointed and will not purchase again .'

In [19]:
amazon.loc[0, "helpful"]

False

In [20]:
ridge_predictions = rdg.predict(X)

In [21]:
ridge_predictions.shape

(91000,)

In [22]:
print(ridge_predictions)

[False False False ... False False False]


In [23]:
# false positives

import random, time

for i in range(0, len(ridge_predictions)):
    if (ridge_predictions[i] == True):
        if (amazon.loc[i, "helpful"] == False):
            if (random.uniform(0, 1) < 0.05):
                print(i)
                print(time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(amazon.loc[i, "Time"])))
                print(amazon.loc[i, "Score"])
                print(amazon.loc[i, "HelpfulnessDenominator"])
                print(amazon.loc[i, "ProductId"])
                print(amazon.loc[i, "Text"])
                print('* * * * * * * * * ')

1529
2008-12-20 00:00:00
5
0
B0006U7T12
We have a wonderful German shepherd in our family.  She's happy and affectionate and a wonderful companion.  Exactly the kind of dog that one enjoys rewarding for her good behavior.<br /><br />And our dog, Asha, seems to look at 7.4 ounce Jumbones as something to which she is entitled.  In short, she loves these treats! She weighs more than she should, so perhaps we are being overindulgent with her, but she loves these so much that it's hard not to provide a supply to last the week for her.<br /><br />For me? I don't know how good these really are.  But, trust me, my dog loves them.
* * * * * * * * * 
5409
2010-09-23 00:00:00
1
21
B002OMB1C8
Mimosa pudica, while fun, is an invasive alien in many countries and toxic to wildlife and grazing animals. Mimosa trees have been banned in Florida. Mimosa trees are banned in many areas because of short lives, branches breaking easily, tendencies to invade and harm native species.<br /><br /><a href="http:/

86568
2012-04-22 00:00:00
5
0
B004F1LNDI
The Stinger Waffles taste great and have been a life saver more than once when I've "bonked" on a long ride or hike. I like the vanilla ones a little better, but both taste great.
* * * * * * * * * 
87079
2009-09-22 00:00:00
5
0
B000SW4NNK
This is a nice small bottle of minced garlic with a wonderful rich, full aroma.  The garlic itself is very mild though.  It has absolutely no BITE!  unlike some fresh garlic bulbs.  This is great for some recipes but others...like hummus... would benefit from a more aggressive tangy taste.  Being prepared and bottled, there are no bulbs to go bad with excessive time and no mess of peeling and mincing.  I really like these Goya minced garlic bottles and highly recommend them to anyone who enjoys a milder garlic flavor. I will be buying them again!
* * * * * * * * * 
88519
2012-08-30 00:00:00
5
0
B004XTDU8I
Badshah Punjabi Garam Masala.<br />Garam Masala is a quintessential ingredient in various dishes of Indian